In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# Корреляция 

![](https://idatassist.com/wp-content/uploads/2017/04/dreamstime_m_37904189-610x461.jpg)

Напомним определение коэффициента корреляции между векторами $x = (x_1, \ldots, x_n)$ и $y = (y_1, \ldots, y_n)$:

$$
  \rho = \frac{\sum_{i=1}^n (x_i - \overline x)(y_i - \overline y)}{\sqrt{\sum_{i=1}^n (x_i - \overline x)^2} \sqrt{\sum_{i=1}^n (y_i - \overline y)^2}}
$$

Более подробное описание есть в лекционных [слайдах](https://drive.google.com/file/d/1pM1NKSXlIj47EM2w5LK_9lX-f6kIK2q_/view).


Для демонстрации эффекта зависимости в данных загрузим датасет с потреблением топлива различными видами транспорта:

In [ ]:
path = 'http://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data'

mpg_data = pd.read_csv(
    path,
    delim_whitespace=True, header=None,
    names = [
        'mpg', 'cylinders', 'displacement',
        'horsepower', 'weight', 'acceleration',
        'model_year', 'origin', 'name'
    ],
    na_values='?',
)
mpg_data.head()

Удалим пропуски:

In [ ]:
mpg_data = mpg_data.dropna()

Посмотрим на коэффициент корреляции между скоростью потребления топлива `mpg` (miles per gallon, галлон $\approx 3.785$ литра) с весом:

In [ ]:
print("Коэффициент корреляции: ", np.corrcoef(mpg_data.mpg, mpg_data.weight)[0, 1])

Наблюдаем сильную отрицательную корреляцию.

Что это значит:
- если вес транспортного средства большой, то одного галлона хватает на не очень большое количество миль
- если транспорт лёгкий, то на одном галлоне он сможет проехать большее расстояние, чем тяжёлый
- т.е. с увеличением веса уменьшается расстояние, которое можно проехать, потратив фиксированное количество топлива

Такую ситуацию можно искусственно смоделировать:
- например, сохраним в переменную значения столбца `acceleration`
- создадим переменную с теми же значениями, но немного зашумлёнными: такое могло произойти при сломанном датчике, использованном для замерения ускорения

In [ ]:
X = mpg_data.acceleration
Y = X + np.random.normal(0, 1, size=len(X))

print("Коэффициент корреляции: ", np.corrcoef(X, Y)[0, 1])

Можно наблюдать положительную корреляцию между векторами:

In [ ]:
print('Correlation of X and Y: ', np.corrcoef(X, Y)[0, 1])

plt.scatter(X,Y)
plt.grid()
plt.xlabel('X Value')
plt.ylabel('Y Value')
plt.show()

Аналогичным образом можно смоделировать отрицательную корреляцию между векторами.

Если корреляция близка к нулю, то между векторами либо нет зависимости, либо она очень слабая. Например, случайные числа никак не взаимосвязаны:

In [ ]:
X = np.random.normal(0, 1, 50)
Y = np.random.normal(0, 1, 50)

print('Correlation of X and Y: ', np.corrcoef(X, Y)[0, 1])

plt.scatter(X,Y)
plt.grid()
plt.xlabel('X Value')
plt.ylabel('Y Value')
plt.show()

Постройте график зависимости для рассмотренных ранее столбцов `mpg` и `weight`:

In [ ]:
# YOUR CODE

___
## Задание 

Рассмотрите другие пары признаков в датасете. Найдите среди них те, которые
- слабо коррелируют друг с другом (коэффициент корреляции по модулю $< 2$)
- имеют сильную положительную корреляцию

Для каждой пары постройте график зависимости, аналогичный предыдущему.

In [ ]:
# YOUR CODE

# Линейная регрессия 

Подробное описание можно найти в лекционных [слайдах](https://drive.google.com/file/d/1Eyedj7ELliNHYVYzZSYvAj8j8YTlrhHE/view).


**TL;DR**

Попытаемся предсказать (объяснить) величину $y$ через набор числовых характеристик $x_1, \ldots, x_n$.
Будем предполагать и надеяться, что величина $y$ не просто зависит от этих характеристик, но и выражается следующим образом:
$$y = \beta + \alpha_1x_1 + \alpha_2x_2+...+\alpha_nx_n + \varepsilon,$$
где $\varepsilon$ -- нормально распределённый шум, а коэффициенты при характеристиках -- неизвестны.

Чтобы в дальнейшем иметь возможность угадывать $y$ только по заданным $x_1, \ldots, x_n$, надо подобрать коэффициенты, близкие к реальным (реальные нам неизвестны):
$$y \approx \hat y = b + a_1x_1 + a_2x_2+...+a_nx_n$$

Подбор параметров $b \approx \beta, a_1 \approx \alpha_1, \ldots, a_n \approx \alpha_n$ происходит за счёт минимизации суммарной ошибки по всем известным объектам:
$$MSE = \sum (y_i - \hat{y}_i)^2$$

___


Начнем с самого простого вида линейной регрессии, когда есть только зависимость от одного признака
$$y=\beta + \alpha x+ \varepsilon,$$
где $\alpha$ -- это коэффициент наклона прямой, $\beta$ -- коэффициент смещения, $\varepsilon$ -- нормально распределённый шум.

Посмотрим ещё раз на зависимость потребления топлива от веса транспортного средства:

In [ ]:
X = mpg_data.weight
Y = mpg_data.mpg

print('Correlation of X and Y: ', np.corrcoef(X, Y)[0, 1])

plt.scatter(X,Y)
plt.grid()
plt.xlabel('weight')
plt.ylabel('mpg')
plt.show()

Попробуем вручную подобрать прямую, которая будет приближать наши данные. Как минимум, по коэффициенту корреляции мы уже видим, что коэффициент $\alpha < 0$.

In [ ]:
X = mpg_data.weight
Y = mpg_data.mpg

# Инициализация коэффициентов.
# Подберите такие значения, чтобы отображаемая прямая хорошо приближала исходные данные.
alpha = -1
beta = 0
Y_hat = alpha * X + beta

print('Correlation of X and Y: ', np.corrcoef(X, Y)[0, 1])

plt.scatter(X,Y)
plt.plot(X, Y_hat, 'r')
plt.grid()
plt.xlabel('weight')
plt.ylabel('mpg')
plt.show()

Конечно, обычно никто не подбирает коэффициенты вручную, для этого есть питон и библиотеки.

Можем использовать модель `LinearRegression` из библиотеки `sklearn`, чтобы найти наилучший вариант прямой, описывающей наши данные:

In [ ]:
from sklearn.linear_model import LinearRegression


X = mpg_data.weight
Y = mpg_data.mpg

model = LinearRegression(fit_intercept=True)

# np.newaxis нужен, чтобы добавить размерность
model.fit(X[:, np.newaxis], Y)

x_predict = np.linspace(1000, 5000, 10)
y_predict = model.predict(x_predict[:, np.newaxis])

plt.plot(X, Y, 'o')
plt.grid()
plt.plot(x_predict, y_predict, 'r')

Посмотрим, получились ли коэффициенты близкими к тому, что мы попытались угадать:

In [ ]:
print("Коэффициент уклона:    ", model.coef_[0])
print("Коэффициент смещения: ", model.intercept_)

Аналогичным образом можно обучить модель на всех признаках, а не только на одном. Но визуализировать это уже будет сложновато, так что понадобится какая-то метрика оценивания качества предсказаний.

![](https://theneural.files.wordpress.com/2011/07/valid2.jpeg)

В анализе данных есть два эпата: обучение модели и валидация. На этапе обучения модель видит и объекты, и целевую переменную. На этих данных настраиваются веса. На этапе валидации в модель подаются объекты, которых не было на эпате обучения, без целевой переменной, и модель уже самостоятельно получает решение.

Метрика `mean_squared_error` или `MSE` оценивает величину ошибки. Она вычисляется отдельно для предсказаний на обучении и на валидации.

---

Для начала подготовим данные:
- выделим в отдельную переменную то, что предсказывается, т.е. столбец `mpg`
- а ещё сохраним все остальные столбцы с числовыми значениями, которые будут нашими признаками

In [ ]:
Y = mpg_data.mpg
X = mpg_data.drop(['mpg', 'name'], axis=1)
X.shape, Y.shape

Теперь поключим необходимые функции для разделения выборки на обучающую и тестовую, а также для оценивания качества модели:

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [ ]:
X_train, X_val, Y_train, Y_val = train_test_split(X, Y)

model = LinearRegression()
model.fit(X_train, Y_train)

Y_train_predicted = model.predict(X_train)
Y_val_predicted = model.predict(X_val)
train_error = mean_squared_error(Y_train_predicted, Y_train)
val_error = mean_squared_error(Y_val_predicted, Y_val)

print('Ошибка на обучении:  \t', train_error)
print('Ошибка на валидации:\t', val_error)
print('Разница:\t\t', val_error - train_error)

___
___


![](https://i.stack.imgur.com/t0zit.png)

Модель может недообучаться, обучаться хорошо и переобучаться.

Визуализируем эффект переобучения. Вновь обратимся к зависимости потребления топлива от веса транспорта.

Напомним, что зависимость у этих величин наблюдалась не совсем линейная:

In [ ]:
X1 = mpg_data.weight
Y = mpg_data.mpg

plt.grid()
plt.plot(X1, Y, 'o')

Попробуем приближать данные разными степенями полинома:
$$
  y(x) = a_0 + a_1x + \ldots + a_nx^n
$$

Поэкспериментируйте с разными значениями степени и проанализируйте результаты:

In [ ]:
# X1 хранит первую степень значений (т.е. просто исходные данные).
X1 = mpg_data.weight
Y = mpg_data.mpg

max_power = 1  # эту переменную можно менять и смотреть, что происходит

variables_list = []
for i in range(1, max_power + 1):
    variables_list.append(X1 ** i)

X = np.column_stack(variables_list)

# Разделите выборку на train и validation с помощью функции train_test_split.
# YOUR CODE

# Определите линейную модель.
model = # YOUR CODE
# Обучите модель на обучающей (!) выборке.
# YOUR CODE

# Предскажите ответы для обучающей и тестовой выборки.
Y_train_predicted = # YOUR CODE
Y_val_predicted = # YOUR CODE

# Подсчёт среднеквадратичной ошибки на обучающей и контрольной выборках.
train_error = mean_squared_error(Y_train_predicted, Y_train)
val_error = mean_squared_error(Y_val_predicted, Y_val)
print('Ошибка на обучении:  \t', train_error)
print('Ошибка на валидации:\t', val_error)
print('Разница:\t\t', val_error - train_error)


f, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 8))
ax1.plot(X_train[:, 0], Y_train, 'o')
ax1.plot(X_train[:, 0], Y_train_predicted, 'o')
ax1.set_ylim(Y.min(), Y.max())
ax1.set_title('Train samples and prediction.')

ax2.plot(X_val[:, 0], Y_val, 'o')
ax2.plot(X_val[:, 0], Y_val_predicted, 'o')
ax2.set_title('Test samples and prediction.')
# comment the line below to see all dots
# ax2.set_ylim(Y.min(), Y.max())

___
___
## Задание

В данном задании будем строить модель линейной регрессии, используя Boston Housing Dataset.

In [ ]:
from sklearn.datasets import load_boston
boston = load_boston()
print(boston.DESCR)

In [ ]:
data = pd.DataFrame(boston.data, columns = boston.feature_names)
data['PRICE'] = boston.target
data.head()

Необходимо найти коэффициент корреляции между двумя различными парами признаков и изобразить эти признаки на графике (как мы делали выше).

In [ ]:
x = # YOUR CODE
y = # YOUR CODE

# YOUR CODE

В дальнейшем, чтобы не смотреть коэффициент корреляции для каждой пары отдельно, можно вывести корреляционную таблицу сразу для всех признаков:

In [ ]:
mpg_data.corr()

In [ ]:
data.corr()

Позже мы научимся её удобно визуализировать.

Теперь предскажем на основе всех признаков цену на жилье (PRICE).

In [ ]:
X = #YOUR CODE
Y = #YOUR CODE

# Разделите выборку на train и validation с помощью функции train_test_split.
# YOUR CODE

# Определите линейную модель.
model = # YOUR CODE
# Обучите модель на обучающей (!) выборке.
# YOUR CODE

# Предскажите ответы для обучающей и тестовой выборки.
Y_train_predicted = # YOUR CODE
Y_val_predicted = # YOUR CODE

# Подсчёт среднеквадратичной ошибки на обучающей и контрольной выборках.
train_error = mean_squared_error(Y_train_predicted, Y_train)
val_error = mean_squared_error(Y_val_predicted, Y_val)
print('Ошибка на обучении:  \t', train_error)
print('Ошибка на валидации:\t', val_error)
print('Разница:\t\t', val_error - train_error)